In [1]:
import pandas as pd
import os
import glob

In [2]:
# path = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), 'data', 'en-de'))
path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'en-de')

In [3]:
os.path.dirname(os.getcwd())

'/Users/theophile/Documents/Masters_ML/NLP/coursework/sentence_level_qe_ffl'

In [4]:
data = [f for f in glob.glob(path + "**/*", recursive=True)]

In [5]:
names=['Val English Source', 'Train scores', 
       'Val scores', 'Val German tranlsation', 
       'Train German translation', 'Test German translation', 
       'Test English source', 'Train English source']

In [6]:
data_frames = [pd.read_table(data[i], names=[names[i]], sep='delimiter', engine='python') for i in range(len(data))]

In [7]:
train = pd.concat((data_frames[7], data_frames[4], data_frames[1]), axis=1)

In [9]:
train.nsmallest(20, "Train scores")

,Train English source,Train German translation,Train scores
3917,"The Mummy, A Handbook of Egyptian Funerary Arc...","The Mummy, A Handbook of Egyptian Funerary Arc...",-8.140713
6975,He vacated the WWA Cruiserweight title in Apri...,Nach seiner Rückkehr zum WWF verließ er im Apr...,-7.481519
2080,"Haroun and S.A. Mourad, Proceedings of Interna...","Haroun and S.A. Mourad, Proceedings of Interna...",-7.148534
6229,Boone Trails Grand Towers Gravois Trail New Ho...,Boone Trails Grand Towers Gravois Trail New Ho...,-7.028717
1846,"Faerie Apocalypse, Jason Franks, IFWG Publishi...","Faerie Apocalypse, Jason Franks, IFWG Publishi...",-6.776486
2163,"Pollock, Griselda, Generations and Geographies...","Pollock, Griselda, Generations and Geographies...",-6.776486
2214,Illustrated Encyclopedia of Woodworking Handto...,Illustrated Encyclopedia of Woodworking Handto...,-6.776486
2576,"""The Great Reform Act of 1832 and British Demo...","""The Great Reform Act of 1832 and British Demo...",-6.776486
2500,"""Radio Birdman"", in The Trouser Press Record G...","""Radio Birdman"", in The Hose Press Record Guid...",-6.629883
2604,"U.S. Army National Infantry Museum, ""Indian Wa...","U.S. Army National Infantry Museum, ""Indian Wa...",-6.624294


In [10]:
train.nlargest(20, "Train scores")

,Train English source,Train German translation,Train scores
6635,132 Slim Williams and his Tennessee Avenue Boy...,132 Slim Williams und seine Tennessee Avenue J...,2.764582
947,"On 12 June 1626, the Commons launched a direct...",Am 12. Juni 1626 startete die Commons einen di...,2.227719
2255,"On the 5th, a northward drift ensued while wel...","Am 5. folgte eine Norddrift, während weit südö...",2.227719
5563,"Soon afterwards, the JMA too downgraded Muifa ...",Bald darauf hat die JMA Muifa zu einem schwere...,2.227719
6808,The sulfate minerals all contain the sulfate a...,"Die Sulfatminerale enthalten alle Sulfatanion,...",2.227719
719,The eastern bobwhite attacks these ants by dig...,"Der Ostbobwhite greift diese Ameisen an, indem...",2.093815
4404,The visibility of prostitution in Laos belies ...,Die Sichtbarkeit der Prostitution in Laos wide...,2.093815
410,Legislators who codified these laws sought to ...,"Gesetzgeber, die diese Gesetze kodifizierten, ...",1.850865
2875,"In the aftermath of the assassination, the Met...",Nach der Ermordung unterdrückten die Metropoli...,1.850865
3330,Confirmation of President Obama's first nomine...,Die Bestätigung des ersten Kandidaten von Präs...,1.850865


In [11]:
train.median()

Train scores    0.164712
dtype: float64

In [14]:
train[train['Train scores']<0]

,Train English source,Train German translation,Train scores
1,"However, a disappointing ninth in China meant ...",Eine enttäuschende Neunte in China bedeutete j...,-0.516656
2,"In his diary, Chase wrote that the release of ...","In seinem Tagebuch, Chase schrieb, dass die Ve...",-2.226388
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,-0.827379
5,They engaged in crossfire at Guamaní River Bri...,Sie verübten Kreuzfeuer an der Guamaní River B...,-0.688501
7,16 Penn State in which redshirt freshman Jim K...,"16 Penn State, in dem Rothemd-Neuling Jim Kell...",-0.493057
...,...,...,...
6990,Classical style with rendered brickwork and sl...,Klassischer Stil mit gerenderten Mauerwerk und...,-0.166602
6992,Those attacks exacted a toll from Beales neigh...,Diese Angriffe forderten eine Maut von Beales ...,-0.074015
6994,"Albany fled to France, then England, where he ...","Albany floh nach Frankreich, dann England, wo ...",-0.209080
6997,"Disco was criticized as mindless, consumerist,...","Disco wurde als geistlos, konsumistisch, überp...",-1.096680
